Data Analysis HW2 - Decision Tree 

In [5]:
import numpy as np # linear algebra
import scipy as sp 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz


ModuleNotFoundError: No module named 'graphviz'

這次選用的 Dataset 為 Heart Disease UCI
透過分析性別、年齡、血壓、膽固醇等指標，結合 Decision Tree
試著預測受測者是否有心血管疾病。

In [2]:
pd_data = pd.read_csv('./heart.csv')
pd_data = pd_data.sample(frac=1).reset_index(drop=True)
pd_train = pd_data.head(250)
pd_test = pd_data.tail(50)
pd_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,54,0,1,132,288,1,0,159,1,0.0,2,1,2,1
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1
2,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
3,57,1,0,110,335,0,1,143,1,3.0,1,1,3,0
4,42,0,0,102,265,0,0,122,0,0.6,1,0,2,1
5,60,1,0,117,230,1,1,160,1,1.4,2,2,3,0
6,60,1,0,145,282,0,0,142,1,2.8,1,2,3,0
7,64,1,3,110,211,0,0,144,1,1.8,1,0,2,1
8,44,1,2,140,235,0,0,180,0,0.0,2,0,2,1
9,65,0,0,150,225,0,0,114,0,1.0,1,3,3,0


In [3]:
pd_train.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,54,0,1,132,288,1,0,159,1,0.0,2,1,2,1
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1
2,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
3,57,1,0,110,335,0,1,143,1,3.0,1,1,3,0
4,42,0,0,102,265,0,0,122,0,0.6,1,0,2,1
5,60,1,0,117,230,1,1,160,1,1.4,2,2,3,0
6,60,1,0,145,282,0,0,142,1,2.8,1,2,3,0
7,64,1,3,110,211,0,0,144,1,1.8,1,0,2,1
8,44,1,2,140,235,0,0,180,0,0.0,2,0,2,1
9,65,0,0,150,225,0,0,114,0,1.0,1,3,3,0


In [4]:
pd_test.tail(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
293,59,1,2,126,218,1,1,134,0,2.2,1,1,1,0
294,53,0,0,130,264,0,0,143,0,0.4,1,0,2,1
295,61,1,0,140,207,0,0,138,1,1.9,2,1,3,0
296,64,1,0,128,263,0,1,105,1,0.2,1,1,3,1
297,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1
298,69,1,2,140,254,0,0,146,0,2.0,1,3,3,0
299,57,1,0,110,201,0,1,126,1,1.5,1,0,1,1
300,58,1,0,128,216,0,0,131,1,2.2,1,3,3,0
301,64,1,0,145,212,0,0,132,0,2.0,1,2,1,0
302,63,0,0,124,197,0,1,136,1,0.0,1,0,2,0


In [5]:
pd_obs = pd_train.sort_values(by=['thalach'], ascending = False)
pd_obs.head(20)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
171,29,1,1,130,204,0,0,202,0,0.0,2,0,2,1
164,54,1,1,192,283,0,0,195,0,0.0,2,1,3,0
156,42,1,2,120,240,1,1,194,0,0.8,0,0,3,1
239,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
197,52,1,3,118,186,0,0,190,0,0.0,1,0,1,1
201,44,1,1,130,219,0,0,188,0,0.0,2,0,2,1
83,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
90,51,1,0,140,261,0,0,186,1,0.0,2,0,2,1
24,45,1,0,115,260,0,0,185,0,0.0,2,0,2,1
38,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1


首先從 14 個